In [2]:
%pip install requests python-dotenv pydub

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import requests
from datetime import datetime
from dotenv import load_dotenv
from pydub import AudioSegment

# ✅ Load environment variables
load_dotenv()

# 🔐 API Keys
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")

# 🌐 API URLs
GEMINI_URL = os.getenv("GEMINI_URL")
MUSICGEN_URL = os.getenv("MUSICGEN_URL")
ELEVENLABS_URL = os.getenv("ELEVENLABS_URL")

# 📁 Create output folder
os.makedirs("outputs", exist_ok=True)


# ✨ 1. Generate lyrics using Gemini
def generate_lyrics(prompt):
    print("🎤 Generating lyrics using Gemini...")

    headers = {"Content-Type": "application/json"}
    body = {
        "contents": [
            {"parts": [{"text": f"Write short emotional song lyrics about: {prompt}. Include verses and a chorus."}]}
        ]
    }

    url = f"{GEMINI_URL}?key={GEMINI_API_KEY}"
    response = requests.post(url, headers=headers, json=body)

    if response.status_code != 200:
        raise Exception(f"❌ Gemini Error: {response.text}")

    lyrics = response.json()["candidates"][0]["content"]["parts"][0]["text"]

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    lyrics_path = f"outputs/lyrics_{timestamp}.txt"
    with open(lyrics_path, "w", encoding="utf-8") as f:
        f.write(lyrics)

    print(f"✅ Lyrics saved: {lyrics_path}")
    return lyrics, timestamp


# 🎼 2. Generate background music with MusicGen
def generate_music(lyrics, timestamp):
    print("🎶 Generating background music...")

    headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}
    payload = {"inputs": f"Instrumental background music for these lyrics: {lyrics}"}

    response = requests.post(MUSICGEN_URL, headers=headers, json=payload)

    if response.status_code != 200:
        raise Exception(f"❌ MusicGen Error: {response.text}")

    music_path = f"outputs/music_{timestamp}.wav"
    with open(music_path, "wb") as f:
        f.write(response.content)

    print(f"✅ Music saved: {music_path}")
    return music_path


# 🎙️ 3. Generate vocals with ElevenLabs
def generate_vocals(lyrics, timestamp):
    print("🗣️ Generating vocals with ElevenLabs...")

    voice_id = "Rachel"  # You can change this to another ElevenLabs voice name
    headers = {
        "xi-api-key": ELEVENLABS_API_KEY,
        "Content-Type": "application/json"
    }
    body = {
        "text": lyrics,
        "voice_settings": {"stability": 0.7, "similarity_boost": 0.8}
    }

    response = requests.post(f"{ELEVENLABS_URL}/{voice_id}/stream", headers=headers, json=body)

    if response.status_code != 200:
        raise Exception(f"❌ ElevenLabs Error: {response.text}")

    vocals_path = f"outputs/vocals_{timestamp}.mp3"
    with open(vocals_path, "wb") as f:
        f.write(response.content)

    print(f"✅ Vocals saved: {vocals_path}")
    return vocals_path


# 🎧 4. Combine vocals + music
def combine_audio(vocals_path, music_path, timestamp):
    print("🎚️ Mixing vocals and music...")

    vocals = AudioSegment.from_file(vocals_path)
    music = AudioSegment.from_file(music_path)

    # Adjust volume balance
    music = music - 5  # make background softer

    combined = music.overlay(vocals)
    final_path = f"outputs/final_song_{timestamp}.mp3"
    combined.export(final_path, format="mp3")

    print(f"🎵 Final Song saved: {final_path}")


# 🚀 Main workflow
if __name__ == "__main__":
    theme = input("🎨 Enter your song theme: ")

    try:
        lyrics, timestamp = generate_lyrics(theme)
        music_path = generate_music(lyrics, timestamp)
        vocals_path = generate_vocals(lyrics, timestamp)
        combine_audio(vocals_path, music_path, timestamp)
        print("\n🌟 Song successfully created! Check the 'outputs' folder.")
    except Exception as e:
        print("\n❌ Error:", e)

E:\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


🎤 Generating lyrics using Gemini...
✅ Lyrics saved: outputs/lyrics_20251030_162517.txt
🎶 Generating background music...

❌ Error: ❌ MusicGen Error: {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
